<a href="https://colab.research.google.com/github/MaribelLuque/SaturdaysAI/blob/master/Borrador_Proyecto_incendios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prediction of forest fires using artificial intelligence

## About

###...Saturdays AI

Saturdays.AI is a non-profit in a mission to empower diverse individuals to learn Artificial Intelligence in a collaborative and project-based way, beyond the conventional education path.

During the first half of the program we learn the foundations of machine-learning and deep-learning by coding exercises on GPU enabled environments in a collaborative way, assisted by facilitators and mentors.

In the second half of the program, we build end2end AI powered prototypes using what we learnt on the "code2learn" phase, working with the dataset and model of our choice, we address a real problem with AI.

SaturdayAIFellows are committed to creating **positive social impact**, open source projects in exchange for their accessible education.

###...This project

Forest fires has become one of the biggest ecological problems suffered by our forests due to the high frequency and intensity it has acquired in recent years.

In Spain there is an annual average of 14,476 fires affecting 108,282.39 hectares of surface area (data from the last ten years 2005-2014). Fuente: Ministerio de Agricultura, Pesca y Alimentación.

The key is **prevention and early detection**. In this project we apply artificial intelligence to predict in real time the probability that fire will start, the intensity of the fire if it occurs and the measures, if any, that should be taken.

###...Our datasets

####*Fire information for resource management system (FIRMS)*
Summary elaborated by Rafa Sánchez
rafael.sanchez.duran@gmail.com

where to find https://firms.modaps.eosdis.nasa.gov/



NASA logo The Fire Information for Resource Management System (FIRMS) distributes Near Real-Time (NRT) active fire data within 3 hours of satellite observation from both the Moderate Resolution Imaging Spectroradiometer (MODIS) and the Visible Infrared Imaging Radiometer Suite (VIIRS).

The active fire / hotspot data can be viewed in FIRMS Fire Map or in NASA’s Worldview , delivered as email alerts or downloaded in the following formats: SHP, KML, TXT, WMS

FIRMS is part of NASA’s Land, Atmosphere Near real-time Capability for EOS (LANCE).


DATASETS:

Active Fire Data  
Download active fire products from the Moderate Resolution Imaging Spectroradiometer (MODIS) (MCD14DL) and Visible Infrared Imaging Radiometer Suite (VIIRS) 375 m (VNP14IMGTDL_NRT) for the last 24, 48 hours and 7 days in shapefile, KML, WMS or text file formats. VIIRS data complement MODIS fire detections but the improved spatial resolution of the 375 m data provides a greater response over fires of relatively small areas. Read more about VIIRS...
Data older than 7 days can be obtained from the Archive Download Tool. Users intending to perform scientific analysis are advised to download the standard (science quality) data.

Please note:

MODIS C6 is available from November 2000 (for Terra) and from July 2002 (for Aqua) to the present.
VIIRS 375 m data are currently available from 20 January 2012 to the present.

TXT https://earthdata.nasa.gov/earth-observation-data/near-real-time/firms/active-fire-data 

Download text files, in CSV format, for the last 24 and 48 hours, and 7 days.
Access daily text files for the last two months via HTTP: https://nrt4.modaps.eosdis.nasa.gov/archive/FIRMS

For MODIS C6 data go to: https://nrt4.modaps.eosdis.nasa.gov/archive/FIRMS/c6

For VIIRS 375m data go to: https://nrt4.modaps.eosdis.nasa.gov/archive/FIRMS/viirs

To keep file sizes to a minimum, the data are provided by region.

####*Climate information*

These data series come from a collection of images from the MODIS satellite (MOD IS/MCD43A4_006_NDVI). We obtain the NDVI (Index of Vegetation in Normarized Differences) and the LST (Land Surface Temperature) values for a specific data range and region of interest using the Google Earth Engine API. To get this collection of images it is necessary to have previously an active GEE account.


-------Maribel: Aquí añadiría alguna explicación de qué es el índice NDVI, qué representa y para qué sirve. Además la intensidad del viento y su dirección también son dos datos que si se pueden descargar de esta misma forma serían de vital utilidad para el modelo.

####*Wind speed and direction from wind turbines*

Wind energy in Spain is a renewable source of electricity in which Spain has been a pioneer.  With 23,484 MW of accumulated power, wind energy has been the second source of electricity generation in Spain in 2018. Spain is the fifth country in the world in terms of installed wind power, after China, the United States, Germany and India. 
In our country wind energy covers 19% of the energy consumed. There are currently 1,123 wind farms installed in 807 municipalities in Spain. 

This geographic scenario is ideal for the use of wind speed and direction records in each wind turbine as additional data for fire prediction.

For this purpose, we will use as a validation dataset for our predictive model, the data collected during the year 2018 from the 21 wind turbines of the PESUR wind farm (Tarifa, Cádiz) provided by Enel Green Power.

##Imports

In [0]:
#Instalamos los paquetes necesarios.
!pip install fastai==0.7.0
!pip install googledrivedownloader
!pip install utm

In [0]:
#Importamos las librerías
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

import pandas as pd
import os
import numpy as np
import utm
import matplotlib.pyplot as plt

##Read data

In [0]:
#Descargamos desde Google Drive los data sets
from google_drive_downloader import GoogleDriveDownloader as gdd
#El dataset de PESUR
gdd.download_file_from_google_drive(file_id='1RuwvvwU8giKIu3ARMsos5N0_19zrHxEj',
                                    dest_path='./data/datos_PESUR_21_2018EDIT.csv',
                                    unzip=False)

#El data set del histórico de incendios de la NASA
!mkdir './data/HISTORICO_NASA/'
gdd.download_file_from_google_drive(file_id='1nks6UP3aZXS2GqKjuj-qD2uFXVILTTwa',
                                    dest_path='./data/HISTORICO_NASA/fire_archive_V1_56830.csv',
                                    unzip=False)

In [0]:
#Descargamos los datos de los 7 últimos días de incendios de la NASA directamente desde el enlace en la web.
!mkdir './data/ACTUAL_NASA/'
PATH = 'data/ACTUAL_NASA/'
os.chdir (PATH)
!wget https://firms.modaps.eosdis.nasa.gov/data/active_fire/viirs/csv/VNP14IMGTDL_NRT_Europe_7d.csv

In [34]:
#Creamos el dataframe de los datos de PESUR.
#Forzamos a que el campo "Fecha" lo importe con el type datetime
os.chdir('/content')
!pwd 
df_pesur = pd.read_csv('./data/datos_PESUR_21_2018EDIT.csv', low_memory=False, parse_dates=["Fecha"]) 

#Vemos el tipo de datos
df_pesur.info()

/content
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52222 entries, 0 to 52221
Data columns (total 6 columns):
Numero_de_serie           52222 non-null int64
Fecha                     52222 non-null datetime64[ns]
Velocidad_de_viento       52222 non-null float64
Posicion_de_la_gondola    52222 non-null int64
Latitud                   52222 non-null float64
Longitud                  52222 non-null float64
dtypes: datetime64[ns](1), float64(3), int64(2)
memory usage: 2.4 MB


In [8]:
#Creamos el dataframe de los datos HISTÓRICOS DE INCENDIOS.
#Forzamos a que el campo "Fecha y hora de adquisición" lo importe con el type datetime
df_hist = pd.read_csv('./data/HISTORICO_NASA/fire_archive_V1_56830.csv', low_memory=False, parse_dates=["acq_date_time"])

#Vemos el tipo de datos
df_hist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105700 entries, 0 to 105699
Data columns (total 13 columns):
latitude         105700 non-null float64
longitude        105700 non-null float64
bright_ti4       105700 non-null float64
scan             105700 non-null float64
track            105700 non-null float64
satellite        105700 non-null object
instrument       105700 non-null object
confidence       105700 non-null object
version          105700 non-null int64
bright_ti5       105700 non-null float64
frp              105700 non-null float64
type             105700 non-null int64
acq_date_time    105700 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(7), int64(2), object(3)
memory usage: 10.5+ MB


In [10]:
#Creamos el dataframe de los datos HISTÓRICOS DE INCENDIOS.
#Forzamos a que el campo "Fecha y hora de adquisición" lo importe con el type datetime
df_actual = pd.read_csv('./data/ACTUAL_NASA/VNP14IMGTDL_NRT_Europe_7d.csv', low_memory=False, parse_dates=["acq_date"]) 

#Vemos el tipo de datos
df_actual.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8549 entries, 0 to 8548
Data columns (total 13 columns):
latitude      8549 non-null float64
longitude     8549 non-null float64
bright_ti4    8549 non-null float64
scan          8549 non-null float64
track         8549 non-null float64
acq_date      8549 non-null datetime64[ns]
acq_time      8549 non-null int64
satellite     8549 non-null object
confidence    8549 non-null object
version       8549 non-null object
bright_ti5    8549 non-null float64
frp           8549 non-null float64
daynight      8549 non-null object
dtypes: datetime64[ns](1), float64(7), int64(1), object(4)
memory usage: 868.3+ KB


##Look at the data

In [0]:
#Definimos la función "display all" que nos va a servir después para ver los datos. 

def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

In [0]:
#Vemos los datos del dataframe de PESUR cómo es
display_all(df_pesur)

In [0]:
#Vemos los datos del dataframe del histórico de incendios cómo es
display_all(df_hist)

In [0]:
#Vemos los datos del dataframe del histórico de incendios cómo es
display_all(df_actual)

In [15]:
#Función tail para mostrar las últimas entradas de datos
display_all(df_pesur.tail())

,Numero_de_serie,Fecha,Velocidad_de_viento,Posicion_de_la_gondola,Latitud,Longitud
52217,781978,2018-12-31 22:50:00,13.7,92,36.045217,-5.589723
52218,781978,2018-12-31 23:00:00,14.3,92,36.045217,-5.589723
52219,781978,2018-12-31 23:10:00,14.1,94,36.045217,-5.589723
52220,781978,2018-12-31 23:20:00,14.1,94,36.045217,-5.589723
52221,781978,2018-12-31 23:30:00,14.0,94,36.045217,-5.589723


In [16]:
#Función tail para mostrar las últimas entradas de datos
display_all(df_hist.tail())

,latitude,longitude,bright_ti4,scan,track,satellite,instrument,confidence,version,bright_ti5,frp,type,acq_date_time
105695,43.53809,-5.83013,342.6,0.39,0.36,N,VIIRS,n,1,286.8,11.0,2,2018-12-31 13:10:00
105696,43.12484,-6.38784,334.7,0.40,0.37,N,VIIRS,n,1,290.0,4.7,0,2018-12-31 13:10:00
105697,42.32085,-1.78886,325.5,0.41,0.37,N,VIIRS,n,1,285.4,1.8,0,2018-12-31 13:10:00
105698,43.52415,-5.73295,336.9,0.39,0.36,N,VIIRS,n,1,287.6,3.8,2,2018-12-31 13:10:00
105699,43.12572,-6.38267,328.6,0.40,0.37,N,VIIRS,n,1,288.0,4.3,0,2018-12-31 13:10:00


In [17]:
#Vemos alguna información estadística de cómo son nuestros datos
display_all(df_hist.describe(include='all').T)

,count,unique,top,freq,first,last,mean,std,min,25%,50%,75%,max
latitude,105700,NaN,NaN,NaN,NaN,NaN,40.6353,2.76681,27.7241,38.8276,41.4529,42.9811,43.7095
longitude,105700,NaN,NaN,NaN,NaN,NaN,-4.38697,3.26609,-18.1504,-6.23711,-5.3497,-1.90458,4.26278
bright_ti4,105700,NaN,NaN,NaN,NaN,NaN,324.197,20.8,208,304.1,326.7,339.5,367
scan,105700,NaN,NaN,NaN,NaN,NaN,0.459929,0.0905928,0.32,0.39,0.43,0.51,0.8
track,105700,NaN,NaN,NaN,NaN,NaN,0.485535,0.119283,0.36,0.38,0.45,0.57,0.78
satellite,105700,1,N,105700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
instrument,105700,1,VIIRS,105700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
confidence,105700,3,n,95945,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
version,105700,NaN,NaN,NaN,NaN,NaN,1,0,1,1,1,1,1
bright_ti5,105700,NaN,NaN,NaN,NaN,NaN,290.809,12.5124,232.8,283.1,289.1,297.2,380


---------------Maribel: Aquí tenemos que incluir la parte de las gráficas que está preparando Rafa.

##Pre-processing

In [18]:
#Bucle para redondear la fecha y hora de adquisición de los datos de histórico de incendios a los 10 minutos más próximos. 
print("Comienzo")
acq_date_time_redon = []
for i in range(df_hist.shape[0]):
    tm=df_hist.acq_date_time[i]
    discard = datetime.timedelta(minutes=tm.minute % 10, 
          seconds=tm.second, 
          microseconds=tm.microsecond) 
    tm -= discard 
    if discard >= datetime.timedelta(minutes=5): 
        tm += datetime.timedelta(minutes=10)
    acq_date_time_redon.append(tm)
print("Final")

Comienzo
Final


In [0]:
#Convierto la lista anterior a un dataframe y lo anexo como columna adicional al dataframe de historico de incendios
df=pd.DataFrame(acq_date_time_redon, columns=['acq_date_time_redon'])
df_hist = df_hist.join(df)

In [20]:
#Vemos que se ha creado correctamente esta nueva columna
df_hist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105700 entries, 0 to 105699
Data columns (total 14 columns):
latitude               105700 non-null float64
longitude              105700 non-null float64
bright_ti4             105700 non-null float64
scan                   105700 non-null float64
track                  105700 non-null float64
satellite              105700 non-null object
instrument             105700 non-null object
confidence             105700 non-null object
version                105700 non-null int64
bright_ti5             105700 non-null float64
frp                    105700 non-null float64
type                   105700 non-null int64
acq_date_time          105700 non-null datetime64[ns]
acq_date_time_redon    105700 non-null datetime64[ns]
dtypes: datetime64[ns](2), float64(7), int64(2), object(3)
memory usage: 11.3+ MB


In [0]:
#Vemos los datos para comprobar que el redondeo se ha hecho correctamente
display_all(df_hist)

In [0]:
#Redondeamos los campos de latitud y longitud en los dos dataframes para poder hacerlos coincidir
df_hist=df_hist.round({'latitude': 0, 'longitude': 0})
df_pesur=df_pesur.round({'Latitud': 0, 'Longitud': 0})

In [23]:
#Vemos los primeros datos
df_pesur.head()

,Numero_de_serie,Fecha,Velocidad_de_viento,Posicion_de_la_gondola,Latitud,Longitud
0,781978,2018-01-01 00:00:00,7.0,303,36.0,-6.0
1,781978,2018-01-01 00:10:00,9.3,307,36.0,-6.0
2,781978,2018-01-01 00:20:00,8.5,309,36.0,-6.0
3,781978,2018-01-01 00:30:00,7.9,318,36.0,-6.0
4,781978,2018-01-01 00:40:00,7.0,318,36.0,-6.0


In [24]:
df_hist.head()

,latitude,longitude,bright_ti4,scan,track,satellite,instrument,confidence,version,bright_ti5,frp,type,acq_date_time,acq_date_time_redon
0,38.0,-5.0,345.0,0.34,0.56,N,VIIRS,n,1,275.8,4.7,0,2012-01-20 02:55:00,2012-01-20 03:00:00
1,43.0,-0.0,335.0,0.44,0.39,N,VIIRS,n,1,287.1,12.5,0,2012-01-20 12:37:00,2012-01-20 12:40:00
2,41.0,2.0,308.9,0.35,0.56,N,VIIRS,n,1,270.8,1.1,2,2012-01-21 02:35:00,2012-01-21 02:40:00
3,40.0,-4.0,299.1,0.41,0.45,N,VIIRS,n,1,273.6,0.9,2,2012-01-21 02:36:00,2012-01-21 02:40:00
4,40.0,-3.0,297.8,0.41,0.45,N,VIIRS,n,1,272.4,0.8,2,2012-01-21 02:36:00,2012-01-21 02:40:00


In [0]:
#Unimos los dos dataframes para crear un dataframe único para poder incorporar al modelo de RandomForest
#Esto es solo una prueba. La idea es unir los dataframes de histórico de incendios y climatológicos
df_hist.merge(df_pesur,left_on=['latitude','longitude','acq_date_time_redon'],right_on=['Latitud','Longitud','Fecha'],validate='m:1')

##Random Forests